In [3]:
!python --version
!python -c "import sys, pprint; pprint.pprint(sys.path)"
!pip list

Python 3.6.9 :: Anaconda, Inc.


['',
 'C:\\Users\\hp\\Anaconda3\\envs\\btp36\\python36.zip',
 'C:\\Users\\hp\\Anaconda3\\envs\\btp36\\DLLs',
 'C:\\Users\\hp\\Anaconda3\\envs\\btp36\\lib',
 'C:\\Users\\hp\\Anaconda3\\envs\\btp36',
 'C:\\Users\\hp\\Anaconda3\\envs\\btp36\\lib\\site-packages',
 'C:\\Users\\hp\\Anaconda3\\envs\\btp36\\lib\\site-packages\\win32',
 'C:\\Users\\hp\\Anaconda3\\envs\\btp36\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\hp\\Anaconda3\\envs\\btp36\\lib\\site-packages\\Pythonwin']
Package              Version  
-------------------- ---------
absl-py              0.8.0    
astor                0.8.0    
attrs                19.1.0   
backcall             0.1.0    
bleach               3.1.0    
certifi              2019.9.11
colorama             0.4.1    
decorator            4.4.0    
defusedxml           0.6.0    
entrypoints          0.3      
gast                 0.3.2    
grpcio               1.16.1   
h5py                 2.9.0    
ipykernel            5.1.2    
ipython              7.8.0  

In [4]:
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate

Using TensorFlow backend.


In [5]:
from model import create_model
from align import AlignDlib
import os
import numpy as np
import cv2
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, accuracy_score


In [6]:
# Model for 128 embeddings from a image
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

In [7]:
# Model Summary
nn4_small2_pretrained.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 48, 48, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [8]:
# Initialize the OpenFace face alignment utility
alignment = AlignDlib('weights/landmarks.dat')

In [9]:
# Funtion to align an image using dlib utility
def align_image(img):
    return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

In [10]:
def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    return img[...,::-1]


In [11]:
class IdentityMetadata():
    def __init__(self, base, name, file):
        # dataset base directory
        self.base = base
        # identity name
        self.name = name
        # image file name
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
    
def load_metadata(path):
    metadata = []
    for i in sorted(os.listdir(path)):
        for f in sorted(os.listdir(os.path.join(path, i))):
            # Check file extension. Allow only jpg/jpeg' files.
            ext = os.path.splitext(f)[1]
            if ext == '.jpg' or ext == '.jpeg':
#                 print(i," ",path," ",f)
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

metadata = load_metadata('images')
# print(metadata[0])

In [12]:
# print(enumerate(metadata))

# To store image embeddings
embedded = np.zeros((metadata.shape[0], 128))

# Process image through the neural net to get 128 embeddings per image
for i, m in enumerate(metadata):    
    img = load_image(m.image_path())
    img = align_image(img)
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    # obtain embedding vector for image
    embedded[i] = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]

In [13]:
embedded.shape

(100, 128)

In [14]:
from sklearn.preprocessing import LabelEncoder
targets = np.array([m.name for m in metadata])

encoder = LabelEncoder()
encoder.fit(targets)

# Numerical encoding of identities
y = encoder.transform(targets)


In [15]:
train_idx = np.arange(metadata.shape[0]) % 2 != 0
test_idx = np.arange(metadata.shape[0]) % 2 == 0

# 50 train examples of 10 identities (5 examples each)
X_train = embedded[train_idx]
# 50 test examples of 10 identities (5 examples each)
X_test = embedded[test_idx]

y_train = y[train_idx]
y_test = y[test_idx]

svc = LinearSVC()

svc.fit(X_train, y_train)

acc_svc = accuracy_score(y_test, svc.predict(X_test))

print(f'SVM accuracy = {acc_svc}')

SVM accuracy = 0.96


In [16]:
# Prediction Code for live Images

In [109]:
font = cv2.FONT_HERSHEY_SIMPLEX

def processFrame(img):
    # Change RGB to BGR for imshow
    origImg = img[...,::-1]
    
    # Get bounding box
    bb = alignment.getLargestFaceBoundingBox(img)
    
    # Align image
    img = align_image(img)
    
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    
    # obtain embedding vector for image
    features = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
    
    # Get the class for prediction
    predClass = svc.predict([features])
    predClass = encoder.inverse_transform(predClass)[0]
    
    # Display the image with bounding box & label
    cv2.putText(origImg,str(predClass),(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
    cv2.rectangle(origImg,(bb.left(),bb.top()),(bb.right(),bb.bottom()),(0,0,255),2)
    cv2.imshow('image',origImg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [110]:
img1 = load_image("images\Arnold_Schwarzenegger\Arnold_Schwarzenegger_0006.jpg")
bb = alignment.getLargestFaceBoundingBox(img1)
processFrame(img1)

[(67, 68) (175, 175)]
